# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_ocr_ensembl"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_ocr_ensembl 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2022-06-03 07:02:05.538688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 07:02:05.921190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42767 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:a3:00.0, compute capability: 8.6


In [5]:
info(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_ocr_ensembl` has 2 classes: negative, positive.

The length of genomic intervals ranges from 71 to 593, with average 326.3452470873675 and median 315.0.

Totally 174756 sequences have been found, 139804 for training and 34952 for testing.


,train,test
negative,69902,17476
positive,69902,17476


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 139804 files belonging to 2 classes.


In [7]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

In [8]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'g', 'c', 'n']

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [10]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [11]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


2022-06-03 07:02:57.425868: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


   1/2185 [..............................] - ETA: 1:59:41 - loss: 0.6663 - binary_accuracy: 0.5781 - f1_score: 0.0000e+00

2022-06-03 07:02:58.845959: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2185/2185 [==============================] - 45s 19ms/step - loss: 0.6217 - binary_accuracy: 0.6546 - f1_score: 0.4922
Epoch 2/10
2185/2185 [==============================] - 39s 18ms/step - loss: 0.5914 - binary_accuracy: 0.6840 - f1_score: 0.5838
Epoch 3/10
2185/2185 [==============================] - 40s 18ms/step - loss: 0.5754 - binary_accuracy: 0.6982 - f1_score: 0.6137
Epoch 4/10
2185/2185 [==============================] - 39s 18ms/step - loss: 0.5647 - binary_accuracy: 0.7067 - f1_score: 0.6299
Epoch 5/10
2185/2185 [==============================] - 39s 18ms/step - loss: 0.5579 - binary_accuracy: 0.7135 - f1_score: 0.6412
Epoch 6/10
2185/2185 [==============================] - 39s 18ms/step - loss: 0.5522 - binary_accuracy: 0.7172 - f1_score: 0.6500
Epoch 7/10
2185/2185 [==============================] - 40s 18ms/step - loss: 0.5482 - binary_accuracy: 0.7201 - f1_score: 0.6554
Epoch 8/10
2185/2185 [==============================] - 39s 18ms/step - loss: 0.5450 - binary_accurac

## Evaluation on the test set

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 34952 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

547/547 [==============================] - 8s 15ms/step - loss: 0.5851 - binary_accuracy: 0.6885 - f1_score: 0.7196


[0.5850780010223389, 0.6884870529174805, 0.7195802927017212]